In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import PredefinedSplit
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor

In [3]:
# Cargar los datos
wind = pd.read_csv('wind_ava.csv.gz', compression='gzip')

# Importante!! Recordar quitar las columnas que no sirven.

In [4]:
# Partición en train-test (outer)
# Importante!! No hacemos shuffle.
# Importante!! El test no lo usamos hasta el final del todo.
train, test = train_test_split(wind, test_size=1000, shuffle=False)

In [6]:
# Podemos verificar que los datos siguen ordenados.
print(f"Fechas de train: {train.iloc[0].datetime} - {train.iloc[-1].datetime}")
print(f"Fechas de test: {test.iloc[0].datetime} - {test.iloc[-1].datetime}")

Fechas de train: 2005-01-02 18:00:00 - 2008-01-30 06:00:00
Fechas de test: 2008-01-30 12:00:00 - 2009-12-31 18:00:00


In [14]:
# Vamos a separar X (predictores) de y (variable a predecir)
X_train = train.drop(['datetime', 'energy'], axis='columns')
y_train = train[['energy']]

In [15]:
# Vemos cómo quedaría una partición usando TimeSeriesSplit
tss = TimeSeriesSplit(n_splits=3)

# Importante!! Esto solo lo hacemos aquí, usando GridSearchCV nos lo ahorramos
for k, data in enumerate(tss.split(X_train, y_train)):
  cv_train, cv_val = data
  print(f"En la iteración {k+1}, los índices son:")
  print(f"  Train: {cv_train[0]} - {cv_train[-1]}")
  print(f"  Val: {cv_val[0]} - {cv_val[-1]}")

En la iteración 1, los índices son:
  Train: 0 - 936
  Val: 937 - 1873
En la iteración 2, los índices son:
  Train: 0 - 1873
  Val: 1874 - 2810
En la iteración 3, los índices son:
  Train: 0 - 2810
  Val: 2811 - 3747


In [16]:
# Vemos cómo quedaría una partición usando PredefinedSplit
# 70 % train (valor -1) y 30 % test (valor 0)
ix = [-1] * round(len(X_train) * 0.7) + [0] * round(len(X_train) * 0.3)
ps = PredefinedSplit(ix)

# Importante!! Esto solo lo hacemos aquí, usando GridSearchCV nos lo ahorramos
for k, data in enumerate(ps.split(X_train, y_train)):
  cv_train, cv_val = data
  print(f"En la iteración {k+1}, los índices son:")
  print(f"  Train: {cv_train[0]} - {cv_train[-1]}")
  print(f"  Val: {cv_val[0]} - {cv_val[-1]}")

En la iteración 1, los índices son:
  Train: 0 - 2623
  Val: 2624 - 3747


In [21]:
# Hacemos una prueba rápida de GridSearchCV
# (Por ejemplo, usando TimeSeriesSplit)
search = GridSearchCV(
    # Probar varios modelos, cuidado, ¡algunos pueden necesitar normalización!
    estimator=DecisionTreeRegressor(),
    # ¡Debería haber más hiperparámetros a probar!
    param_grid={'max_depth': [2, 5, 10]},
    # Recordemos que era un TimeSeriesSplit con 3 folds.
    cv=tss
)

# Entrenamos los modelos (comparamos las diferentes alternativas usando HPO)
search.fit(X_train, y_train)

# Vemos el mejor modelo
search.best_estimator_

DecisionTreeRegressor(max_depth=5)